In [2]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Read all the needed datasets
dataset들 읽고 합치기

In [24]:
import glob

input_file = r'.\data' # csv파일들이 있는 디렉토리 위치
output_file = r'.\data\Harry Potter.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'Harry Potter *')) # glob함수로 sales_로 시작하는 파일들을 모은다
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    df = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
    df.columns = ['Character','Sentence'] # index를 
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다
dataCombine = pd.concat(allData, axis=0, ignore_index=True) # 리스트의 내용을 수직으로 병합. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인데스를 빼려면 False로 설정

harry=pd.read_csv('./data/Harry Potter.csv')
harry

,Character,Sentence
0,Dumbledore,"I should've known that you would be here, Prof..."
1,McGonagall,"Good evening, Professor Dumbledore."
2,McGonagall,"Are the rumors true, Albus?"
3,Dumbledore,"I'm afraid so, professor."
4,Dumbledore,The good and the bad.
...,...,...
4920,HERMIONE,"How fast is it, Harry?"
4921,HARRY,Lumos.
4922,HARRY,I solemnly swear that I am up to no good.
4923,HARRY,Mischief managed.


In [20]:
harry.groupby('Sentence').nunique().sort_values(by='Sentence',ascending='False')

ValueError: 'Sentence' is both an index level and a column label, which is ambiguous.

# Stemming
morph analysis 기법.

In [38]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps=PorterStemmer()

harry=pd.read_csv('./data/Harry Potter.csv')
print(harry)

trainPotter = harry['Character'].str.replace('[^a-zA-Z ]', '')
print(trainPotter)

tokenData = nltk.word_tokenize(str(trainPotter))
print(tokenData)

stemData=[]
for w in tokenData:
    tempData=[]
    tempData=ps.stem(w)
    stemData.append(tempData)
    
print(stemData)

                                               Sentence
0                                              Sentence
1     I should've known that you would be here, Prof...
2                   Good evening, Professor Dumbledore.
3                           Are the rumors true, Albus?
4                             I'm afraid so, professor.
...                                                 ...
4923                             How fast is it, Harry?
4924                                             Lumos.
4925          I solemnly swear that I am up to no good.
4926                                  Mischief managed.
4927                                               Nox.

[4928 rows x 1 columns]
[nltk_data] Downloading package punkt to
[nltk_data]     D:\Users\kimdu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     D:\Users\kimdu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-d

KeyError: 'Character'

In [6]:
# data 정제하기

harry1['Sentence'] = harry1['Sentence'].str.replace('[^a-zA-Z ]', '') #알파벳과 공백을 제외하고 모두 제거